This Notebook evaluates a convergence study for the 1D Shock-Acoustic-Wave interaction problems as presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [32]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESF;
Init();


Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Databases loaded: 
Capacity: 0
Count: 0



Error: System.ApplicationException: Already called.
   at BoSSS.Application.BoSSSpad.BoSSSshell.InitTraceFile() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 217
   at BoSSS.Application.BoSSSpad.BoSSSshell.Init() in C:\experimental\public\src\L4-application\BoSSSpad\BoSSSshell.cs:line 104
   at Submission#33.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

init workflow


In [62]:
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D_ConvergenceStudy2");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


Project name is set to 'XESTSF_ShockAcousticInteraction1D_ConvergenceStudy2'.
Default Execution queue is chosen for the database.
Opening existing database '\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy2'.


{ Session Count = 15; Grid Count = 15; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D_ConvergenceStudy2 }

In [63]:
var dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path;
var database = OpenOrCreateDatabase(dbPath);
var dt = database.Sessions.Last().CreationTime;
dt


2024-05-05 16:14:32Z

In [35]:
var sessions = database.Sessions;
sessions

#0: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/7/2024 12:50:01 PM	c30bcf0c...
#1: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8*	5/7/2024 12:49:34 PM	e430d224...
#2: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/7/2024 12:49:19 PM	0c1f4c55...
#3: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/3/2024 10:56:08 AM	fd93cb11...
#4: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/3/2024 10:53:19 AM	481bb6ef...
#5: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	5/3/2024 10:54:40 AM	2ea2daa4...
#6: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax4_1sinu

In [64]:
var aaa = new MatrixAssembler(3);
var si = database.Sessions.Pick(0);
var ti = si.Timesteps;
si.ID

08534bdf-f1e0-4aa3-8bf6-e662632f8583

## Choose which runs to plot

In [275]:
var sessions=database.Sessions.Where(s => (s.Name.Contains("sP0.5") &&s.Name.Contains("ampneg1")&&s.Name.Contains("amppos0")&& !s.Name.Contains("241"))); // the one plotted in the publication
sessions.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
sessions


#0: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:14:14 PM	54ce6b78...
#1: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:13:59 PM	18f4b6f9...
#2: XESTSF_AcousticWave1D	AW_p3_xCells16_tCells16_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:13:43 PM	17613dea...
#3: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/5/2024 4:14:32 PM	e3f53a43...


# A Helper Function to obtain different plot formats

In [276]:
using BoSSS.Solution.Tecplot;
using System.IO;
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

## Residual History

In [277]:

using BoSSS.Solution.Tecplot;
using System.IO;
//Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
var allEE = new List<List<double>>();
var plot = new Plot2Ddata();
int count=0;
foreach(var si in sessions){
    var enthalpyerrors = new List<double>();
    IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
    plot.AddDataGroup("||r|| - "+ si.Name,lts.TimeStepNumbers,lts.ResHistory, GetFormat(count));
    count++;
    plot.AddDataGroup("EnRes_"+si.Name,lts.TimeStepNumbers,lts.EnResHistory, GetFormat(count));
    count++;
}

plot.Xlabel="Iteration";
plot.LogY = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:800,yRes:500)

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -14 
 
 
 
 
 10 -13 
 
 
 
 
 10 -12 
 
 
 
 
 10 -11 
 
 
 
 
 10 -10 
 
 
 
 
 10 -9 
 
 
 
 
 10 -8 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||r|| - AWp3xCells61tCells61tMax41sinusMach1.5sP0.5wP0.9ampneg1E-05amppos0wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M53.9,83.5 L67.8,293.6 L81.7,388.7 L95.7,389.4 L109.6,391.8 L123.5,392.7 L137.4,392.7 L151.4,393.6
 L165.3,393.6 L179.2,393.6 L193.1,393.6 L207.1,393.6 L221.0,393.6 L234.9,393.6 L248.8,393.6 L262.8,274.3
 L276.7,372.8 L290.6,374.5 L304.5,372.7 L318.5,373.3 L332.4,375.5 L346.3,376.5 L360.2,376.8 L374.2,377.1
 L388.1,377.6 L402.0,321.3 L415.9,363.6 L429.8,363.9 L443.8,366.3 L457.7,366.7 L471.6,367.3 L485.5,368.1
 L499.5,368.2 L513.4,368.6 L527.3,368.7 L541.2,316.7 L555.2,351.3 L569.1,352.6 L583.0,353.6 L596.9,353.2
 L610.9,354.7 L624.8,355.1 L638.7,355.0 L652.6,356.0 L666.6,355.9 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResAWp3xCells61tCells61tMax41sinusMach1.5sP0.5wP0.9ampneg1E-05amppos0wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M53.9,72.3 L67.8,78.6 L81.7,78.6 L95.7,78.6 L109.6,78.6 L123.5,78.6 L137.4,78.6 L151.4,78.6
 L165.3,78.6 L179.2,78.6 L193.1,78.6 L207.1,78.6 L221.0,78.6 L234.9,78.6 L248.8,78.6 L262.8,97.4
 L276.7,97.4 L290.6,97.4 L304.5,97.4 L318.5,97.4 L332.4,97.4 L346.3,97.4 L360.2,97.4 L374.2,97.4
 L388.1,97.4 L402.0,111.2 L415.9,111.2 L429.8,111.2 L443.8,111.2 L457.7,111.2 L471.6,111.2 L485.5,111.2
 L499.5,111.2 L513.4,111.2 L527.3,111.2 L541.2,104.2 L555.2,104.2 L569.1,104.2 L583.0,104.2 L596.9,104.2
 L610.9,104.2 L624.8,104.2 L638.7,104.2 L652.6,104.2 L666.6,104.2 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r|| - AWp3xCells31tCells31tMax41sinusMach1.5sP0.5wP0.9ampneg1E-05amppos0wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M53.9,89.2 L67.8,299.0 L81.7,400.2 L95.7,402.6 L109.6,403.0 L123.5,403.9 L137.4,403.6 L151.4,404.3
 L165.3,404.5 L179.2,405.0 L193.1,405.1 L207.1,405.1 L221.0,405.2 L234.9,405.2 L248.8,405.2 L262.8,405.2
 L276.7,405.2 L290.6,277.1 L304.5,383.3 L318.5,383.8 L332.4,383.6 L346.3,386.6 L360.2,387.8 L374.2,388.3
 L388.1,388.6 L402.0,388.5 L415.9,389.0 L429.8,302.5 L443.8,372.7 L457.7,373.7 L471.6,374.4 L485.5,374.1
 L499.5,374.5 L513.4,373.9 L527.3,376.6 L541.2,378.0 L555.2,377.5 L569.1,309.7 L583.0,360.3 L596.9,359.8
 L610.9,361.8 L624.8,361.9 L638.7,361.2 L652.6,363.2 L666.6,362.3 L680.5,364.4 L694.4,363.7 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 EnResAWp3xCells31tCells31tMax41sinusMach1.5sP0.5wP0.9ampneg1E-05amppos0wLTruewSTruewL0.8 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M53.9,78.0 L67.8,83.1 L81.7,83.1 L95.7,83.1 L109.6,83.1 L123.5,83.1 L137.4,83.1 L151.4,83.1
 L165.3,83.1 L179.2,83.1 L193.1,83.1 L207.1,83.1 L221.0,83.1 L234.9,83.1 L248.8,83.1 L262.8,83.1
 L276.7,83.1 L290.6,91.7 L304.5,91.7 L318.5,91.7 L332.4,91.7 L346.3,91.7 L360.2,91.7 L374.2,91.7
 L388.1,91.7 L402.0,91.7 L415.9,91.7 L429.8,104.7 L443.8,104.7 L457.7,104.7 L471.6,104.7 L485.5,104.7
 L499.5,104.7 L513.4,104.7 L527.3,104.7 L541.2,104.7 L555.2,104.7 L569.1,98.7 L583.0,98.7 L596.9,98.7
 L610.9,98.7 L624.8,98.7 L638.7,98.7 L652.6,98.7 L666.6,9

helper function to evaluate the XDG field along a x-ray for a specific time

In [278]:
(double[] tVals, double[]yVals) GetPlotAlongXRay(double tMin,double tMax, double x, int nRef,string field,ITimestepInfo ti){
//double shockPosition=0.5;
var p1= new double[] {x,tMin};
var p2=new double[] {x,tMax};

//timesteps to compare

// # use p_per from BoSSS 
var pEnd =ti.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] tVals= GenericBlas.Linspace(tMin,tMax,nRef);
return(tVals,yVals);
}


helper function to compute the error metrics (i.e. the deviation of each amplitude from the exact amplification/reduction factor)

In [304]:
List<double> ComputeAmpErrors(double tMin,double tMax, double xMin, double xMax, int xRef, int tRef,string field,ITimestepInfo ti, bool isMax,double exact_amp_fac){
    if(true){
        var xVals= GenericBlas.Linspace(xMin,xMax,xRef);
        var maxminVals = new List<double>();
        foreach(var x in xVals ){
            (double[] tVals, double[] yVals)=GetPlotAlongXRay(tMin,tMax,x,tRef,field,ti);
            if(isMax){
                maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Max())/exact_amp_fac));
            }else{
                maxminVals.Add(Math.Abs((exact_amp_fac-yVals.Min())/exact_amp_fac));
            }
        }
        return maxminVals;
    }else{
        var pEnd =ti.GetField(field);
        var fullcm= CellMask.GetFullMask(pEnd.GridDat);
        var ba= new bool[fullcm.ItemEnum.Count()];
        
        var bb = new BoSSS.Platform.Utils.Geom.BoundingBox(new double[,] {{0,0},{1,1}});
        foreach(var cell in fullcm.ItemEnum){
            pEnd.GridDat.iGeomCells.GetCellBoundingBox(cell, bb);
            var min = bb.Min;
            if(!isMax){
                if( min[0]>=xMin && min[0]<=xMax){
                    ba[cell]=true;
                };
            }else{
                if( min[0]>=xMin && min[0]<=xMax ){
                    ba[cell]=true;
                };
            }

        }
        var cm =new CellMask(pEnd.GridDat,new BitArray(ba));
        var valMin=new double[cm.ItemEnum.Count()];
        var valMax=new double[cm.ItemEnum.Count()];
        pEnd.GetCellwiseExtremalValues(valMin, valMax,cm);
        List<double> maxminVals;
        if(isMax){
            maxminVals=new List<double> {Math.Abs((exact_amp_fac-valMax.Max())/exact_amp_fac)};
        }else{
            maxminVals=new List<double> {Math.Abs((exact_amp_fac-valMin.Min())/exact_amp_fac)};
        }
        return maxminVals;
    }
    
}

var eps=0.000001;
double xMin=1.8 +eps;double xMax=2.0-eps; int xRef=5; 
double tMin =0 + eps;int tRef=800;
GenericBlas.Linspace(xMin,xMax,xRef)

[ 1.800001, 1.8500005, 1.9, 1.9499995, 1.999999 ]

In [305]:
var ses= sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:14:14 PM	54ce6b78...
#1: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:13:59 PM	18f4b6f9...
#2: XESTSF_AcousticWave1D	AW_p3_xCells16_tCells16_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:13:43 PM	17613dea...
#3: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/5/2024 4:14:32 PM	e3f53a43...


choose the sessions

they must be reordered for the plot to work...

In [306]:
var ses= sessions;
ses = ses.OrderBy(si => si.GetGrids().Pick(0).NumberOfCells);
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells16_tCells16_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:13:43 PM	17613dea...
#1: XESTSF_AcousticWave1D	AW_p3_xCells31_tCells31_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:13:59 PM	18f4b6f9...
#2: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	5/5/2024 4:14:14 PM	54ce6b78...
#3: XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/5/2024 4:14:32 PM	e3f53a43...


Number of cells

In [307]:
int[] Cells = ses.Select(si => si.GetGrids().Pick(0).NumberOfCells).ToArray();
Cells

[ 256, 961, 3721, 14641 ]

helper arrays

In [308]:
int[] degs = new int[] {0,1,2,3};
//degs = new int[] {0};
var errors_array= MultidimensionalArray.Create(degs.Count(),ses.Count());


Function used to calculate the EOCs

In [309]:
double ApproxSlope(List<double> errors, List<double> cellSize) {
    var slopes = new List<double>();
    double EOC;
    for (int i = 0; i < errors.Count - 1; i++) {
        double ratio = errors[i] / errors[i + 1];
        double slope = Math.Log(ratio) / Math.Log(cellSize[i] / cellSize[i + 1]);
        slopes.Add(slope);
    }
    if (slopes.Count > 0) {
        EOC = slopes.Average();
    } else {
        EOC = 0;
    }
    return EOC;
}

Function used to compute the ideal convergence plot

In [310]:
List<double> GetIdealConvPlot(List<double> errors,List<double> cellSize, int p){
        var idealErrors = new List<double>();

    // Calculate ideal errors for each cell size
    for (int i = 0; i < cellSize.Count; i++)
    {
        double idealError = errors[0] * Math.Pow(cellSize[i] / cellSize[0], p + 1);
        idealErrors.Add(idealError);
    }

    return idealErrors;
} 

compute the exact analytical pressure amplification/reduction factor for both cases investigated

In [311]:
// Given variables
var MBaseL = 1.5; // Base Mach number on the left
var gamma = 1.4; //Specific heat ratio

// Calculate right Mach number (Mach_R)
var MBaseR  = Math.Sqrt((1 + ((gamma-1)/2) * MBaseL*MBaseL) / (gamma * MBaseL*MBaseL - (gamma-1)/2));

// Upstream fast acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrL = ((1+MBaseL)*(1+MBaseL)) / (1+2*MBaseR+1/MBaseL/MBaseL) * (1 - ((gamma-1)/(gamma+1)) * (1-(1/MBaseL))* (1-(1/MBaseL)));

// Downstream slow acoustic wave amplification coefficient
var delta_pPrR_over_delta_pPrMiR = -(1- 2*MBaseR +1 / MBaseL/MBaseL) / (1+ 2*MBaseR +1 / MBaseL/ MBaseL);
(delta_pPrR_over_delta_pPrL,delta_pPrR_over_delta_pPrMiR)

Item1,2.154925878735875
Item2,-0.01484811194806477


Create the convergence plot

In [319]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(ses.Count(),degs.Length +1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(ses.Count(),degs.Length+1);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =0;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        Console.Write($"P={p},sess={iSess}");
        //extract the relevant field and timestep Index
        var timesteps = si.Timesteps;
        var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
        if(rTSIndex !=-1){
            IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
            var fields = lts.Fields.ToList();
            cells.Add(Math.Sqrt(fields[0].Basis.GridDat.iGeomCells.GetCellVolume(0)));
            //extract AmpError
            if(si.Name.Contains("sP0.5")){
                double exact_amp_fac=delta_pPrR_over_delta_pPrMiR*1e-05; double tMin=0.0 +eps;double tMax=4.0-eps;
                List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,false,exact_amp_fac);
                //errors_array[p,iSess]=ampErrors.Average();
                errors.Add(ampErrors.Average());
                Console.WriteLine($" sp0.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");
            }else{
                double exact_amp_fac=delta_pPrR_over_delta_pPrL*1e-05; double tMin=0.0 +eps;double tMax=1.0-eps;
                List<double> ampErrors= ComputeAmpErrors(tMin,tMax, xMin, xMax, xRef,tRef,"p_per",lts,true,exact_amp_fac);
                //errors_array[p,iSess]=ampErrors.Average();
                errors.Add(ampErrors.Average());
                Console.WriteLine($" sp1.5, SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");
            }
            
            
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

P=0,sess=0 sp0.5, SQPIter:12/42, h: 0.21650635094610965, error: 0.8844508569845226
P=0,sess=1 sp0.5, SQPIter:16/46, h: 0.11174521339154046, error: 0.8012041462555745
P=0,sess=2 sp0.5, SQPIter:14/44, h: 0.056788551067832045, error: 0.6746474092443782
P=0,sess=3 sp0.5, SQPIter:17/47, h: 0.02862893896808062, error: 0.5062298511148559
P=1,sess=0 sp0.5, SQPIter:22/42, h: 0.21650635094610965, error: 0.3611746040080134
P=1,sess=1 sp0.5, SQPIter:26/46, h: 0.11174521339154046, error: 0.09179437551412153
P=1,sess=2 sp0.5, SQPIter:24/44, h: 0.056788551067832045, error: 0.010814024886505538
P=1,sess=3 sp0.5, SQPIter:27/47, h: 0.02862893896808062, error: 0.0009087350770774879
P=2,sess=0 sp0.5, SQPIter:32/42, h: 0.21650635094610965, error: 0.030160707473669234
P=2,sess=1 sp0.5, SQPIter:36/46, h: 0.11174521339154046, error: 0.0024985078443422277
P=2,sess=2 sp0.5, SQPIter:34/44, h: 0.056788551067832045, error: 0.0002954352077144685
P=2,sess=3 sp0.5, SQPIter:37/47, h: 0.02862893896808062, error: 1.7841

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -5 
 
 
 
 
 10 -4 
 
 
 
 
 10 -3 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [318]:
EOCs

[ 0.43127239256287186, 1.3693011326695022, 1.4458395303623761, 1.527320167604093 ]

plot it

In [314]:
// using BoSSS.Solution.Tecplot;
// using System.IO;
// Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));
// IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) si.Timesteps.Last()).GetInternal();
// //double[] TStoPlot = new double[] {0,5,10,15,20}; // specify selected Iterations
// var allTs = (List<double>) lts.TimeStepNumbers; 
// var N=40;
// var TStoPlot= allTs.Where(ts => ts % N == 0); // plot every N
// foreach(var si in ses){
//     foreach(var timestep in si.Timesteps){
//         if(TStoPlot.Contains((double) timestep.TimeStepNumber.MajorNumber)){
//                 IDTTimeStepInfo lts= (IDTTimeStepInfo)((TimestepProxy) timestep).GetInternal();
//                 var SF = lts.GetShadowFields();
//                 var texplot = new Tecplot(timestep.Fields.Pick(0).GridDat, 2);
//                 texplot.PlotFields( si.Name + "_" + lts.TimeStepNumber,0.0,timestep.Fields);
//         }
//     }
// }

In [315]:

var siRef= ses.Pick(3);
siRef

XESTSF_AcousticWave1D	AW_p3_xCells121_tCells121_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8*	5/5/2024 4:14:32 PM	e3f53a43...

In [316]:
var plot = new Plot2Ddata();
var EOCs= new List<double>();
public PlotFormat GetFormat(int count){
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Circle, PointTypes.OpenCircle, PointTypes.Diamond, PointTypes.OpenDiamond, PointTypes.UpperTriangle, PointTypes.OpenUpperTriangle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue,LineColors.Blue, LineColors.Black, LineColors.Black, LineColors.Red, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointType = PointTypes.OpenCircle;
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;
    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}
var solRef= (XDGField) siRef.Timesteps.Last().GetField("p_per");
var sol= solRef.CloneAs();
int count=0;
MultidimensionalArray errorsAndCells=MultidimensionalArray.Create(ses.Count(),degs.Length +1);
MultidimensionalArray errorsAndCellsIdeal=MultidimensionalArray.Create(ses.Count(),degs.Length+1);
foreach(int p in degs){
    var cells= new List<double>();
    var errors= new List<double>();
    for(int iSess =1;iSess < ses.Count();iSess++){
        var si = ses.Pick(iSess);
        if(p<3  || iSess<3){
            Console.Write($"P={p},sess={iSess}");
            //extract the relevant field and timestep Index
            var timesteps = si.Timesteps;
            var rTSIndex = timesteps.LastIndexWhere(t => t.GetField("rho").Basis.Degree == p);
            if(rTSIndex !=-1){
                IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)timesteps.Pick(rTSIndex)).GetInternal();
                var p_per = (XDGField) lts.GetField("p_per");
                cells.Add(Math.Sqrt(p_per.Basis.GridDat.iGeomCells.GetCellVolume(0)));
                sol.Clear();
                sol.GetSpeciesShadowField("L").ProjectFromForeignGrid(1.0,p_per.GetSpeciesShadowField("L"));
                sol.GetSpeciesShadowField("R").ProjectFromForeignGrid(1.0,p_per.GetSpeciesShadowField("R"));
                errors.Add(solRef.L2Error(sol)); 
                
                Console.WriteLine($" SQPIter:{rTSIndex}/{si.Timesteps.Count-1}, h: {cells.Last()}, error: {errors.Last()}");           
            }
        }
    }
    errorsAndCells.SetColumn(0,cells);
    errorsAndCellsIdeal.SetColumn(0,cells);
    errorsAndCells.SetColumn(p+1,errors);
    errorsAndCellsIdeal.SetColumn(p+1,GetIdealConvPlot(errors,cells,p));
    EOCs.Add( ApproxSlope(errors,cells));
    plot.AddDataGroup($"P={p}",cells,errors, GetFormat(count));
    var fmt = GetFormat(count);
    fmt.PointSize=0.01;
    fmt.DashType=DashTypes.Dashed;

    plot.AddDataGroup($"ideal {p+1}",cells,GetIdealConvPlot(errors,cells,p), fmt);
    count++;
}
errorsAndCells.SaveToTextFile("ConvergenceStudyErrors.txt");
errorsAndCellsIdeal.SaveToTextFile("ConvergenceStudyErrorsIdeal.txt");
plot.Xlabel="Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:500)

P=0,sess=1 SQPIter:16/46, h: 0.11174521339154046, error: 6.370372118761987E-06
P=0,sess=2 SQPIter:14/44, h: 0.056788551067832045, error: 4.972465851792367E-06
P=0,sess=3 SQPIter:17/40, h: 0.02862893896808062, error: 3.538934096954572E-06
P=1,sess=1 SQPIter:26/46, h: 0.11174521339154046, error: 1.3159152848117959E-06
P=1,sess=2 SQPIter:24/44, h: 0.056788551067832045, error: 4.89411475445144E-07
P=1,sess=3 SQPIter:27/40, h: 0.02862893896808062, error: 2.0403815993365087E-07
P=2,sess=1 SQPIter:36/46, h: 0.11174521339154046, error: 3.73666085451476E-07
P=2,sess=2 SQPIter:34/44, h: 0.056788551067832045, error: 1.494930448319361E-07
P=2,sess=3 SQPIter:37/40, h: 0.02862893896808062, error: 5.2126390398214056E-08
P=3,sess=1 SQPIter:46/46, h: 0.11174521339154046, error: 1.9976973549105987E-07
P=3,sess=2 SQPIter:44/44, h: 0.056788551067832045, error: 7.104707845958208E-08
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Not

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -9 
 
 
 
 
 10 -8 
 
 
 
 
 10 -7 
 
 
 
 
 10 -6 
 
 
 
 
 10 -5 
 
 
 
 
 10 -2 
 
 
 
 
 10 -1 
 
 
 
 
 10 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Cells 
 
 
 
 
 P=0 
 
 
 P=0 
 
 
 
 
 
 
 
 
 
 ideal 1 
 
 
 ideal 1 
 
 
 
 
 
 
 
 
 
 P=1 
 
 
 P=1 
 
 
 
 
 
 
 
 
 
 ideal 2 
 
 
 ideal 2 
 
 
 
 
 
 
 
 
 
 P=2 
 
 
 P=2 
 
 
 
 
 
 
 
 
 
 ideal 3 
 
 
 ideal 3 
 
 
 
 
 
 
 
 
 
 P=3 
 
 
 P=3 
 
 
 
 
 
 
 
 
 ideal 4 
 
 
 ideal 4

In [317]:
EOCs

[ 0.43127239256287186, 1.3693011326695022, 1.4458395303623761, 1.527320167604093 ]